# PART I: TITLE CREATION

In [71]:
import pandas as pd
import os
import numpy as np
import re
from collections import Counter
import spacy
from spacy.language import Language
from langdetect import detect

In [72]:
def title_maker(year):
    
    #list of all the port names in the files
    port_names = [
    "Aigun",
    "Harbin District",
    "Hunchun & Lungchingtsun",
    "Moukden",
    "Antung",
    "Dairen",
    "Newchwang",
    "Chinwangtao & Tientsin",
    "Lungkow & Chefoo",
    "Kiaochow",
    "Chungking & Wanhsien",
    "Ichang",
    "Shasi",
    "Changsha",
    "Yochow",
    "Hankow",
    "Kiukiang",
    "Wuhu",
    "Nanking",
    "Chinkiang",
    "Shanghai",
    "Soochow",
    "Hangchow",
    "Ningpo",
    "Wenchow",
    "Santuao",
    "Foochow",
    "Amoy",
    "Swatow",
    "Canton",
    "Kowloon",
    "Lappa",
    "Kongmoon",
    "Samshui",
    "Wuchow",
    "Nanning",
    "Kiungchow",
    "Pakhoi",
    "Lungchow",
    "Mengtsz",
    "Szemao",
    "Tengyueh"
]
    #list of all possible table types in the files 
    all_table_types = ['Revenue','Shipping','Reports To The Customs','Values','Imports','Exports','Inland Transit' ,'Treasure','Passenger Traffic','Special']
    
    
    folder_path = '/Users/hinaljajal/Downloads/' + str(year) + '/csv_and_txt'
    
    files_list = os.listdir(folder_path)
    
    #dataframe with names of all the files in the folder 
    df = pd.DataFrame(files_list,columns=['file_name'])
    
    df = df.sort_values(by="file_name",ascending=True).reset_index().drop(columns=['index'])
    
    #get file type (csv or txt)
    df["file_type"] = df["file_name"].str[-3:]
    
    df["file_name"] = df["file_name"].str[:-4]
    
    df["title_port"] = np.nan
    
    df['title_port'] = df['title_port'].astype(object)

    df["title_table_type"] = np.nan
    
    df['title_table_type'] = df['title_table_type'].astype(object)
    
    #list that will contain the page numbers of all content pages
    content_pages = []
    
    #loops through each txt file in the folder
    for index, row in df.iterrows():
        
        if row['file_type'] == 'txt':
            
            file_path = '/Users/hinaljajal/Downloads/' + str(year) + '/txt/' + str(df.loc[index]['file_name'])+'.txt'
            file = open(file_path, 'r')
            Lines = file.readlines()
            
            #removes trailing characters from each line in the file
            lines_list = [line.strip('\n').strip('.').title() for line in Lines]
            
            #removes roman numerical characters from the beginning of all lines (this is for the port name titles)  
            combinations_to_remove = [r"I\.?[-—─-]", r"Ii\.?[-—─-]",r"Iii\.?[-—─-]",r"Iv\.?[-—─-]",r"V\.?[-—─-]",r"Vi\.?[-—─-]",r"Vii\.?[-—─-]",r"Viii\.?[-—─-]",r"Ix\.?[-—─-]"]
            pattern = '|'.join(combinations_to_remove)
            lines_list = [re.sub(pattern, "", line) for line in lines_list]
            
            #convert lists to sets
            file_set = set(lines_list)
            all_ports = set(port_names)
            all_table_types = set(all_table_types)
            
            #check whether any port names occur in the file
            common_elements_title_port = file_set.intersection(all_ports)

            # Check if exactly one port name appears
            if len(common_elements_title_port) == 1:
                #if yes, this file is the cover/starting page for the port whose name appears  
                df.at[index, 'title_port'] = list(common_elements_title_port)[0]

            #check if any table type titles occur in the file 
            common_elements_title_table = file_set.intersection(all_table_types)
            
            #check if this file contains the word "Contents" once 
            if len(file_set.intersection({"Contents"})) == 1:
                #if yes, this page is the contents page
                content_pages.append(row['file_name'])
            #else, check if exactly one table type name appears in this file 
            elif len(common_elements_title_table) == 1:
                #if yes, this is the cover/starting page for that table type
                df.at[index, 'title_table_type'] = list(common_elements_title_table)[0]
            #else, check if exactly two different table type names appear in this file 
            elif len(common_elements_title_table) == 2:
                #if yes, this is the cover/starting page for that table type
                df.at[index, 'title_table_type'] = ','.join(common_elements_title_table)
    
    #forward fills the title column, ie. every page takes the value of the last valid/occuring title port name  
    df['title_port'] = df['title_port'].fillna(method='ffill')
    
    df.loc[0]['title_port']=df.loc[1]['title_port']
    
    #within the same port name, every page takes the value of the last valid/occuring table type   
    df["new_table_type"] = df.groupby('title_port')['title_table_type'].fillna(method="ffill")
    
    #marks all the content pages
    df.loc[df['file_name'].isin(content_pages), 'new_table_type'] = "Contents"
    
    df = df.drop(columns={"title_table_type"})
    df = df.rename(columns={"new_table_type":"title_table_type"})
    
    df["year"] = year
    df["file_path"] = df["file_name"] +"."+ df["file_type"]
    
    #files that come before the table types are categorized as "Notes"   
    df = df.fillna("Notes")
    
    df["title"] = df["title_port"]+ "-" +df["title_table_type"]
    
    return df[["title_port","title_table_type","year","file_path","title"]]
    

In [73]:
df_1923 = title_maker(1923)

In [74]:
df_1924 = title_maker(1924)

# PART II: NLP ENTITIES CREATION

### II(a) Create entities for all csv files

In [80]:
def nlp_entities_maker_csv(year):
    
    folder_path_csv = '/Users/hinaljajal/Downloads/'+str(year)+'/csv'
    files_list = os.listdir(folder_path_csv)
    csv_files_df = pd.DataFrame(files_list,columns=['file_name'])
    csv_files_df['nlp_entities'] = '[]'

    for index, row in csv_files_df.iterrows():

        df = pd.read_csv(folder_path_csv + '/' +str(row["file_name"]))
        
        #method 1 works only on "Special" files that have names of specific goods 
        try:
            df["DESCRIPTION OF GOODS."] = df["DESCRIPTION OF GOODS."].fillna("").astype(str)
            
            # Convert the description of goods column into a single string
            combined_text = ' '.join(df["DESCRIPTION OF GOODS."].tolist())

            #NER model
            nlp = spacy.load('en_core_web_sm')

            doc = nlp(combined_text)

            # Extract and filter named entities with the desired label ('WORK_OF_ART')
            entities = [ent.text for ent in doc.ents if ent.label_ == 'WORK_OF_ART']

            # Count the occurrences of each named entity
            entities_counter = Counter(entities)

            # Get the top 7 most important 'WORK_OF_ART' entities based on frequency
            top_7_work_of_art_entities = entities_counter.most_common(7)

            csv_files_df.at[index, 'nlp_entities'] = top_7_work_of_art_entities
        
        #method 2: works on rest of the csv files
        except KeyError: 
            #converts all columns to string and concatenate their values into a single string
            combined_text = ' '.join(df.astype(str).apply(lambda x: ' '.join(x), axis=1).tolist())

            #includs the column names as potential named entities
            column_names = list(df.columns)
            combined_text += ' '.join(column_names)

            #NER model
            nlp = spacy.load('en_core_web_sm')

            doc = nlp(combined_text)

            #excludes entities with digits and entities containing 'NaN'
            entities = [ent.text for ent in doc.ents if not any(char.isdigit() for char in ent.text) and 'nan' not in ent.text.lower()]

            #counts the occurrences of each entity
            entity_counts = Counter(entities)

            #the 7 most common entities
            most_common_entities = entity_counts.most_common(7)

            csv_files_df.at[index, 'nlp_entities'] = most_common_entities
            
            csv_files_df = csv_files_df.rename(columns={"file_name":"file_path"})
            
    return csv_files_df

### II(b) Create entities for all txt files

In [87]:
def nlp_entities_maker_txt(year):

    @Language.component("filter_non_english_entities")
    def filter_non_english_entities(doc):
        filtered_ents = []
        for ent in doc.ents:
            if all(token.is_alpha for token in ent):
                filtered_ents.append(ent)
        doc.ents = filtered_ents
        return doc
    
    #NER model
    nlp = spacy.load('en_core_web_sm')
    
    #remove non-english entitites
    nlp.add_pipe("filter_non_english_entities", last=True)

    folder_path_txt = '/Users/hinaljajal/Downloads/' + str(year) + '/txt'

    files_list = os.listdir(folder_path_txt)
    txt_files_df = pd.DataFrame(files_list,columns=['file_name'])
    txt_files_df['nlp_entities'] = '[]'

    for index, row in txt_files_df.iterrows():
        file_path = folder_path_txt + '/' + str(row['file_name'])
        # Read the content of the text file
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()

        doc = nlp(text)

        #excludes entities with digits and entities containing 'NaN'
        entities = [ent.text for ent in doc.ents if not any(char.isdigit() for char in ent.text) and 'nan' not in ent.text.lower()]

        #counts the occurrences of each entity
        entity_counts = Counter(entities)

        #the 7 most common entities
        most_common_entities = entity_counts.most_common(7)

        txt_files_df.at[index, 'nlp_entities'] = most_common_entities
        
        txt_files_df = txt_files_df.rename(columns={"file_name":"file_path"})
        
    return txt_files_df



# FINAL FUNCTION 

In [88]:
# years = [1923, 1924, 1925, 1926, 1927, 1928]
years = [1923]
def spreadsheet_maker(year):
#     for year in years: 
    csv_files_df = nlp_entities_maker_csv(year)
    txt_files_df = nlp_entities_maker_txt(year)
    nlp_entities_year = pd.concat([txt_files_df, csv_files_df], axis=0)
    df_titles_year = title_maker(year)
    all_cols_year = pd.merge(df_titles_year,nlp_entities_year,on="file_path",how="left")
    return all_cols_year

In [89]:
spreadsheet_maker(1923)

,title_port,title_table_type,year,file_path,title,nlp_entities
0,Aigun,Notes,1923,43675355.txt,Aigun-Notes,"[(December Quarter, 2), (China, 1), (one, 1), ..."
1,Aigun,Notes,1923,43675356.txt,Aigun-Notes,"[(Shanghai, 3), (Limited, 3), (HARVARD, 1), (L..."
2,Aigun,Contents,1923,43675357.txt,Aigun-Contents,"[(the Maritime Customs, 4), (the Maritime Cust..."
3,Aigun,Notes,1923,43675358.txt,Aigun-Notes,"[(Blagovestchensk, 3), (the year, 2), (Russian..."
4,Aigun,Notes,1923,43675359.txt,Aigun-Notes,"[(Aigun, 2), (Customs, 2), (the year, 2), (Oct..."
...,...,...,...,...,...,...
2089,Tengyueh,Inland Transit,1923,43676683.txt,Tengyueh-Inland Transit,"[(Chinese Goods, 1), (Transit Certificate, 1),..."
2090,Tengyueh,Inland Transit,1923,43676684.txt,Tengyueh-Inland Transit,[]
2091,Tengyueh,Inland Transit,1923,43676685.txt,Tengyueh-Inland Transit,"[(個 位 二, 1)]"
2092,Tengyueh,Inland Transit,1923,43676686.csv,Tengyueh-Inland Transit,"[(INDIAN, 1), (JAPANESE, 1), (Marks, 1), (Roub..."
